In [2]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import json

In [3]:
i = 1
news = []

while (page := requests.get(f'https://www.hse.ru/news/page{i}.html')).status_code != 404:
    soup = BeautifulSoup(page.text)
    news.extend([
        x.a['href'] for x in soup.find_all('h2')
    ])
    
    i += 1
    print(i, end='\t')

    # Comment the following code to scrap all news

    if i == 200:
        break

2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	25	26	27	28	29	30	31	32	33	34	35	36	37	38	39	40	41	42	43	44	45	46	47	48	49	50	51	52	53	54	55	56	57	58	59	60	61	62	63	64	65	66	67	68	69	70	71	72	73	74	75	76	77	78	79	80	81	82	83	84	85	86	87	88	89	90	91	92	93	94	95	96	97	98	99	100	101	102	103	104	105	106	107	108	109	110	111	112	113	114	115	116	117	118	119	120	121	122	123	124	125	126	127	128	129	130	131	132	133	134	135	136	137	138	139	140	141	142	143	144	145	146	147	148	149	150	151	152	153	154	155	156	157	158	159	160	161	162	163	164	165	166	167	168	169	170	171	172	173	174	175	176	177	178	179	180	181	182	183	184	185	186	187	188	189	190	191	192	193	194	195	196	197	198	199	200	

In [4]:
news = ['https://www.hse.ru' + x if x[0] == '/' else x for x in news]

In [4]:
with open('news.json', 'w', encoding='utf8') as f:
    json.dump(news, f, ensure_ascii=False, indent=2)

In [5]:
news_texts = []

for link in tqdm(news):
    html = requests.get(link).text
    soup = BeautifulSoup(html)
    pars = soup.find('div', {'class': 'post'})
    if not pars:
        continue
    pars = pars.find_all('p')
    news_texts.append(
        '\n'.join(par.text for par in pars)
    )

  0%|          | 0/1990 [00:00<?, ?it/s]

In [6]:
with open('news_texts.json', 'w', encoding='utf8') as f:
    json.dump(news_texts, f, ensure_ascii=False, indent=2)

In [7]:
verbs = [
    'думать',
    'знать',
    'лгать',
    'обнаружить',
    'обнаруживать',
    'подумать',
    'постесняться',
    'признавать',
    'признать',
    'пытаться',
    'рассказать',
    'рассказывать',
    'сказать',
    'сожалеть',
    'солгать',
    'стесняться',
    'суметь',
    'считать',
    'хотеть'
]

In [8]:
from nltk.tokenize import sent_tokenize, word_tokenize
from pymorphy2 import MorphAnalyzer
import pandas as pd
from collections import defaultdict

m = MorphAnalyzer()

In [9]:
data = []

forms = {}

for text in tqdm(news_texts):
    sents = sent_tokenize(text)
    for i, sent in enumerate(sents):
        for word in word_tokenize(sent):
            if word in forms:
                lemma = forms.get(word)
            else:
                lemma = m.parse(word)[0].normal_form
                forms[word] = lemma
            if lemma in verbs:
                data.append({
                    'Source': 'HSE News',
                    'Verb': lemma,
                    'Embedding': None,
                    'PreContext': ' '.join(sents[i - 2:i]),
                    'Target': sent,
                    'PostContext': ' '.join(sents[i + 1:i + 3])
                })

  0%|          | 0/1814 [00:00<?, ?it/s]

In [10]:
df = pd.DataFrame(data)

In [11]:
df = df.reindex(columns = df.columns.tolist() + [
    'MatTense',
    'MatSubjPers',
    'MatSubjNum',
    'MatAspect',
    'SubTense',
    'SubSubjPers',
    'SubSubjNum',
    'SubAspect',
    'Conjunction'])

In [12]:
df.to_excel('new_input.xlsx', index=False)